# Building with LangChain
Primero, importamos Ollama desde langchain_community.llms, que es una abstracción dentro de LangChain para trabajar con modelos de lenguaje  
A continuación, creamos una instancia de Ollama especificando el modelo que queremos utilizar. En este caso, estamos inicializando llama2, que es una versión del modelo de lenguaje LLaMA

In [9]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

Una vez inicializado, el objeto llm puede ser utilizado para realizar inferencias, generar texto, o cualquier otra tarea compatible con el modelo LLaMA en el contexto de aplicaciones desarrolladas con LangChain.

In [20]:
llm.invoke("What is blockchain?")

'\nBlockchain is a decentralized, digital ledger that records transactions and data in a secure and transparent manner. It is the underlying technology behind cryptocurrencies such as Bitcoin and Ethereum, but its potential uses go far beyond digital currency. Here are some key things to know about blockchain:\n\n1. Decentralized: Blockchain is a decentralized system, meaning that there is no central authority controlling it. Transactions are recorded on the blockchain through a network of computers, called nodes, that work together to validate and add new transactions to the ledger.\n2. Distributed Ledger: A blockchain is a distributed ledger, meaning that it is stored on multiple computers across a network. This makes it more secure than a traditional centralized database, as there is no single point of failure.\n3. Immutable: Once a transaction is recorded on the blockchain, it cannot be altered or deleted. This ensures the integrity and security of the data, as any attempts to mani

Importamos ChatPromptTemplate y StrOutputParser desde el módulo langchain_core. Estos componentes son esenciales para la creación de prompts estructurados y para la interpretación de las respuestas del modelo de lenguaje  
A continuación, definimos una plantilla de prompt utilizando ChatPromptTemplate. Esta plantilla se utiliza para generar prompts que se enviarán al modelo de lenguaje, asegurando consistencia y estructura en las interacciones

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

output_parser = StrOutputParser()

Se encadenan varios componentes juntos para formar una única secuencia de procesamiento en LangChain. Esta cadena combina la creación de prompts, el modelo de lenguaje y el análisis de la salida en un flujo de trabajo coherente y eficiente.  
El operador | es utilizado aquí para conectar los componentes de manera funcional, pasando la salida de uno como entrada al siguiente. Este enfoque facilita la modularidad y la reusabilidad de los componentes, permitiendo:

Mantenimiento Sencillo: Cada componente puede ser actualizado o reemplazado independientemente sin afectar el resto de la cadena.
Flexibilidad: Se pueden añadir o remover componentes fácilmente para experimentar con diferentes configuraciones o para ajustar el comportamiento del sistema.

In [10]:
chain = prompt | llm | output_parser

In [11]:
chain.invoke({"input": "What is blockchain?"})

"\nAs a world-class technical documentation writer, I'm here to explain Blockchain in a clear and concise manner! 🚀\n\nBlockchain is a decentralized digital ledger that records transactions across multiple computers within a network. It allows for secure, transparent, and tamper-proof data storage and sharing, without the need for intermediaries like banks or governments.\n\nThink of it as a digital canvas where transactions are painted with unique codes, creating an unalterable record of events. Each transaction is verified by a network of computers working together, making it nearly impossible to alter or falsify the data. This distributed consensus mechanism is what gives Blockchain its immense security and reliability.\n\nThe most well-known application of Blockchain is Bitcoin, a decentralized digital currency that enables peer-to-peer transactions without the need for traditional financial intermediaries like banks. However, Blockchain's potential uses go far beyond just currency

# Retrieval Chain
Primero, importamos WebBaseLoader del módulo langchain_community.document_loaders. Este componente está diseñado para facilitar la carga de documentos desde URLs específicas

In [21]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.bancolombia.com/centro-de-ayuda/preguntas-frecuentes/como-funciona-la-billetera-digital-bpay#:~:text=Es%20un%20sistema%20electr%C3%B3nico%20que,D%C3%A9bito%2C%20y%20la%20sincronizaci%C3%B3n%20con")

docs = loader.load()

Inicializamos embeddings basados en el modelo LLaMA (Language Model Architecture for Many Applications) usando la biblioteca LangChain. Los embeddings son representaciones vectoriales de texto que capturan aspectos semánticos y pueden ser utilizados en una variedad de tareas de procesamiento de lenguaje natural.

In [22]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

Utilizamos RecursiveCharacterTextSplitter, que es una herramienta de LangChain diseñada para dividir textos en partes más pequeñas basadas en caracteres, lo cual es útil para procesar documentos grandes o para mejorar la granularidad en tareas de procesamiento de texto  
FAISS se utiliza para crear una base de datos de vectores que permite búsquedas rápidas y eficientes. Inicializamos FAISS con los documentos divididos y los embeddings generados por el modelo LLaMA

In [23]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Configuramos un prompt específico que será usado para estructurar la entrada al modelo de lenguaje. Este prompt incluye tanto el contexto como la pregunta específica del usuario

In [27]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Responde la siguiente pregunta en español basado en el contexto proporcionado:

<context>
{context}
</context>

Pregunta: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

Primero, vamos a demostrar cómo ejecutar la cadena de documentos que hemos configurado previamente para responder preguntas específicas utilizando un contexto detallado. Esta funcionalidad es esencial para aplicaciones donde la precisión de las respuestas depende fuertemente del contexto proporcionado.

In [28]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Qué  puedo hacer con Bpay?",
    "context": [Document(page_content="Es un sistema electrónico que te permite hacer transacciones financieras de forma virtual, como pagos, transferencias, lectura con QR, consulta de saldo, recargas en efectivo o desde otros bancos, retiros en corresponsales y cajeros, sin contar con el uso de Tarjetas de Crédito o Débito, y la sincronización con Bancolombia A la mano")]
})

'\nCon Bpay, podrás realizar diversas transacciones financieras de forma virtual y sin el uso de tarjetas de crédito o débito. Algunas de las posibilidades que tienes son:\n\n* Hacer pagos a empresas o particulares de manera rápida y segura.\n* Realizar transferencias de dinero a cuentas bancarias de otros bancos o a personas específicas.\n* Leer tu saldo y ver la balanza de tus transacciones en tiempo real.\n* Recargar tu saldo en efectivo desde otros bancos o cajeros automáticos.\n* Retirar dinero de tus cuentas bancarias en otros bancos o cajeros automáticos.\n* Sincronizar tus transacciones con Bancolombia, permitiéndote una mayor visibilidad y control sobre tus finanzas.'

Ahora configuramos una cadena de recuperación de documentos, que combina las capacidades de búsqueda de documentos con la generación de respuestas basadas en el contexto recuperado. Esta funcionalidad es crucial para sistemas de información y Q&A donde la precisión depende de la habilidad para encontrar y utilizar el contexto más relevante

In [29]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [30]:
response = retrieval_chain.invoke({"input": "Qué es Bpay?"})
print(response["answer"])

BPay es una plataforma digital que permite a los usuarios realizar pagos y transferencias de dinero de forma segura, rápida y conveniente. Es una billetera digital que se puede utilizar para realizar pagos en línea o en tienda, y también se puede usar para enviar dinero a personas o empresas.

BPay es una plataforma que ofrece una variedad de servicios, incluyendo:

* Pagos en línea o en tienda: los usuarios pueden realizar pagos en línea o en tienda utilizando su tarjeta de crédito o débito asociada con su cuenta de BPay.
* Transferencias de dinero: los usuarios pueden enviar dinero a personas o empresas utilizando la función de transferencia de BPay.
* Pagos en tiempo real: los usuarios pueden realizar pagos en tiempo real utilizando la función de pagos en tiempo real de BPay.
* Administración de cuentas: los usuarios pueden administrar sus cuentas y ver su historial de pagos y transferencias.

BPay es una plataforma segura y confiable, que utiliza tecnología de cifrado avanzada para

# Conversation Retrieval Chain
La cadena que hemos creado hasta ahora solo puede responder preguntas únicas. Uno de los principales tipos de aplicaciones LLM son los chatbots.Para esto vamos configurar una cadena de recuperación que toma en cuenta el historial de la conversación para buscar información relevante. Esta funcionalidad es especialmente útil en aplicaciones de diálogo donde la coherencia y relevancia del contexto a lo largo del tiempo son cruciales.

In [52]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Dada la conversación anterior, genere una search query para buscar información relevante para la conversación en español.")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

Configuramos el historial de la conversación utilizando instancias de HumanMessage y AIMessage, que representan mensajes del usuario y respuestas de la IA, respectivamente. Esto ayuda a preservar el flujo y contexto de la conversación

In [53]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Qué es Bpay?"), AIMessage(content="Es un sistema electrónico que te permite hacer transacciones financieras de forma virtual, como pagos, transferencias, lectura con QR, consulta de saldo, recargas en efectivo o desde otros bancos, retiros en corresponsales y cajeros, sin contar con el uso de Tarjetas de Crédito o Débito, y la sincronización con Bancolombia A la mano.")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "cuáles son sus funciones?"
})

[Document(page_content='¿Cómo funciona la Billetera digital (BPay)?\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n{}\n\n\n\n\n\n\n\n\n\nZ7_OHHGG4G0P0FKE06MQ68SML15L0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPersonas\n\n\n\nNegocios\n\n\n\nCorporativos\n\n\n\nNegocios especializados \n\n\n\n\n\nNegocios en Colombia\n\n\nBanca de Inversión\n                Bancolombia\n\n\n\nFiduciaria Bancolombia\n\n\n\nLeasing Bancolombia\n\n\n\nRenting Colombia\n\n\n\nValores Bancolombia\n\n\n\nFactoring Bancolombia\n\n\n\nAsset Management\n\n\n\n\n\nSufi\n\n\n\n\n\n\n\n\n\nEntidades en el exterior\n\n\nBancolombia Miami (USA)\n\n\n\nValores Banistmo\n\n\n\nSucursal Panamá\n\n\n\nBancolombia Panamá\n\n\n\nBancolombia Puerto\n                Rico\n\n\n\nBanistmo\n\n\n\nBanco Agrícola\n\n\n\nBAM (Banco Agromercantil\n                de\n                Guatemala)\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTu360\n\n\n\nBlog\n\n\n\n\n\n\n\nTransparencia\n\n\nConsumidor\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nInicio\n\n\n\nNecesida

In [56]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Responda las preguntas del usuario en español según el siguiente contexto:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [57]:
chat_history = [HumanMessage(content="Qué es Bpay?"), AIMessage(content="Es la billetera virtual de Bancolombia!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "¿Cuáles son sus tarifas?"
})

{'chat_history': [HumanMessage(content='Qué es Bpay?'),
  AIMessage(content='Es la billetera virtual de Bancolombia!')],
 'input': '¿Cuáles son sus tarifas?',
 'context': [Document(page_content='¿Cómo funciona la Billetera digital (BPay)?\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n{}\n\n\n\n\n\n\n\n\n\nZ7_OHHGG4G0P0FKE06MQ68SML15L0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPersonas\n\n\n\nNegocios\n\n\n\nCorporativos\n\n\n\nNegocios especializados \n\n\n\n\n\nNegocios en Colombia\n\n\nBanca de Inversión\n                Bancolombia\n\n\n\nFiduciaria Bancolombia\n\n\n\nLeasing Bancolombia\n\n\n\nRenting Colombia\n\n\n\nValores Bancolombia\n\n\n\nFactoring Bancolombia\n\n\n\nAsset Management\n\n\n\n\n\nSufi\n\n\n\n\n\n\n\n\n\nEntidades en el exterior\n\n\nBancolombia Miami (USA)\n\n\n\nValores Banistmo\n\n\n\nSucursal Panamá\n\n\n\nBancolombia Panamá\n\n\n\nBancolombia Puerto\n                Rico\n\n\n\nBanistmo\n\n\n\nBanco Agrícola\n\n\n\nBAM (Banco Agromercantil\n                de\n           